# packages

In [23]:
import pandas as pd
from dask.distributed import Client
import dask.dataframe as dd
import dask.array as da
import pandas as pd
import glob
import os

# dask config

In [24]:
dask_config = {'n_workers':4, 'memory_limit':'5GB', 'threads_per_worker':1,
               'processes': True, 'dashboard_address':':8889'}

In [25]:
client = Client(**dask_config)

# df initial analysis

In [5]:
df = pd.read_parquet('data/capture01.parquet')

In [6]:
df.head()

,No.,tcp length,tcp stream,udp stream,src port,dst port,timestamp,delta time,Source,Destination,Protocol,Length,Info,expert info
0,1,0.0,0.0,NaN,51413.0,19063.0,"2023-07-28 21:21:38,450421",0.000000,142.165.14.87,192.168.100.4,TCP,54,"51413 > 19063 [FIN, ACK] Seq=1 Ack=1 Win=409...",Note
1,2,0.0,0.0,NaN,19063.0,51413.0,"2023-07-28 21:21:38,450718",0.000297,192.168.100.4,142.165.14.87,TCP,54,19063 > 51413 [ACK] Seq=1 Ack=2 Win=512 Len=0,None
2,3,NaN,NaN,0.0,57621.0,57621.0,"2023-07-28 21:21:38,454832",0.004114,192.168.100.4,192.168.100.255,UDP,86,57621 > 57621 Len=44,None
3,4,0.0,1.0,NaN,34069.0,19081.0,"2023-07-28 21:21:38,455025",0.000193,154.16.192.127,192.168.100.4,TCP,54,"34069 > 19081 [RST, ACK] Seq=1 Ack=1 Win=0 L...",Warning
4,5,0.0,2.0,NaN,20482.0,19083.0,"2023-07-28 21:21:38,455025",0.000000,174.114.62.147,192.168.100.4,TCP,54,20482 > 19083 [ACK] Seq=1 Ack=1 Win=255 Len=0,None


In [7]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 400000 entries, 0 to 399999
Data columns (total 14 columns):
 #   Column       Non-Null Count   Dtype  
---  ------       --------------   -----  
 0   No.          400000 non-null  int64  
 1   tcp length   55121 non-null   float64
 2   tcp stream   55170 non-null   float64
 3   udp stream   290422 non-null  float64
 4   src port     345615 non-null  float64
 5   dst port     345615 non-null  float64
 6   timestamp    400000 non-null  object 
 7   delta time   400000 non-null  float64
 8   Source       400000 non-null  object 
 9   Destination  400000 non-null  object 
 10  Protocol     400000 non-null  object 
 11  Length       400000 non-null  int64  
 12  Info         399845 non-null  object 
 13  expert info  11213 non-null   object 
dtypes: float64(6), int64(2), object(6)
memory usage: 42.7+ MB


In [ ]:
df.Source.value_counts()

In [9]:
df.Source.nunique()

2625

In [10]:
df.Destination.nunique()

5664

In [ ]:
df.Destination.value_counts()

In [14]:
df.Protocol.value_counts()

Protocol
UDP           263241
IPv6           54349
TCP            50203
BT-DHT         13109
DNS             5631
TLSv1.2         2664
DNPv100         2265
QUIC            2256
DPPv0           2013
BitTorrent      1135
TLSv1.3         1078
ICMP             861
DNPv33           469
ICMPv6           372
LSD              153
SSDP              32
MDNS              26
ENIP              16
DNPv65            16
OCSP              13
IGMPv3            11
DNPv110            9
SSL                6
DNPv50             5
DNPv114            5
DNPv94             5
HTTP               5
DNPv49             5
DNPv104            4
WireGuard          4
LLMNR              4
DNPv124            4
DNPv17             3
RTPproxy           3
DNPv52             3
DNPv103            3
DNPv15             2
UAUDP              2
DNPv28             1
DNPv55             1
DNPv63             1
DNPv13             1
PNIO               1
DNPv48             1
DNPv25             1
SliMP3             1
GRE                1
RSL 

In [13]:
import random
import socket
import struct

def generate_random_ipv6_address():
    # Generate a random 128-bit number, which is the length of an IPv6 address.
    random_128_bit_number = random.getrandbits(128)

    # Convert the 128-bit number into a format that can be used as an IPv6 address.
    # This format is 16 bytes long, which is the same length as an IPv6 address.
    random_128_bit_number_as_bytes = random_128_bit_number.to_bytes(16, byteorder='big')

    # Convert the 16-byte value into an IPv6 address.
    random_ipv6_address = socket.inet_ntop(socket.AF_INET6, random_128_bit_number_as_bytes)

    return random_ipv6_address

# Test the function.
print(generate_random_ipv6_address())

d917:659a:e32c:22bb:fa12:e398:6ab0:9c68


In [14]:
import random

def generate_random_ipv4_address():
    # Generate four random numbers between 0 and 255, which is the range for each part of an IPv4 address.
    random_parts = [str(random.randint(0, 255)) for _ in range(4)]

    # Combine the four parts with dots to create the IPv4 address.
    random_ipv4_address = ".".join(random_parts)

    return random_ipv4_address

# Test the function.
print(generate_random_ipv4_address())

12.4.58.12


# csvs to parquets

In [14]:
# get a list of all csv files in the directory

csv_directory = 'data'
csv_files = glob.glob(os.path.join(csv_directory, '*.csv'))

# iterate through the list of csv files
for csv_file in csv_files:
    # read the csv file
    csv_data = pd.read_csv(csv_file, encoding='Latin-1')

    # define the parquet file path
    parquet_file = os.path.splitext(csv_file)[0] + '.parquet'

    # write the csv data to a parquet file
    csv_data.to_parquet(parquet_file, engine='pyarrow')

# read parquet files to ddf

In [4]:
ddf = dd.read_parquet('data/capture*.parquet')  

In [31]:
ddf

,No.,tcp length,tcp stream,udp stream,src port,dst port,timestamp,delta time,Source,Destination,Protocol,Length,Info,expert info
npartitions=142,,,,,,,,,,,,,,
,int64,float64,float64,float64,float64,float64,object,float64,object,object,object,int64,object,object
,...,...,...,...,...,...,...,...,...,...,...,...,...,...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
,...,...,...,...,...,...,...,...,...,...,...,...,...,...
,...,...,...,...,...,...,...,...,...,...,...,...,...,...


In [135]:
ddf.head()

,No.,tcp length,tcp stream,udp stream,src port,dst port,timestamp,delta time,Source,Destination,Protocol,Length,Info,expert info
0,1,0.0,0.0,NaN,51413.0,19063.0,"2023-07-28 21:21:38,450421",0.000000,142.165.14.87,192.168.100.4,TCP,54,"51413 > 19063 [FIN, ACK] Seq=1 Ack=1 Win=409...",Note
1,2,0.0,0.0,NaN,19063.0,51413.0,"2023-07-28 21:21:38,450718",0.000297,192.168.100.4,142.165.14.87,TCP,54,19063 > 51413 [ACK] Seq=1 Ack=2 Win=512 Len=0,None
2,3,NaN,NaN,0.0,57621.0,57621.0,"2023-07-28 21:21:38,454832",0.004114,192.168.100.4,192.168.100.255,UDP,86,57621 > 57621 Len=44,None
3,4,0.0,1.0,NaN,34069.0,19081.0,"2023-07-28 21:21:38,455025",0.000193,154.16.192.127,192.168.100.4,TCP,54,"34069 > 19081 [RST, ACK] Seq=1 Ack=1 Win=0 L...",Warning
4,5,0.0,2.0,NaN,20482.0,19083.0,"2023-07-28 21:21:38,455025",0.000000,174.114.62.147,192.168.100.4,TCP,54,20482 > 19083 [ACK] Seq=1 Ack=1 Win=255 Len=0,None


In [10]:
len(ddf)

56477457

# dask df initial analysis

In [5]:
unique_src = ddf['Source'].unique().compute()
unique_dest = ddf['Destination'].unique().compute()

In [6]:
unique_src = unique_src.tolist()
unique_dest = unique_dest.tolist()

In [7]:
unique_src.extend(unique_dest)

In [8]:
s_unique_all = pd.Series(list(set(unique_src)))

In [9]:
l_unique_all = list(s_unique_all)
len(l_unique_all)

77118

In [ ]:
l_unique_all[:20]

In [11]:
local_ips = s_unique_all[(s_unique_all.str.startswith('192.168'))].reset_index(drop = True)
l_local_ips = local_ips.tolist()
local_ips

0       192.168.100.7
1     192.168.100.255
2       192.168.144.2
3       192.168.144.6
4       192.168.100.2
5      192.168.105.38
6       192.168.100.1
7      192.168.144.42
8      192.168.100.12
9      192.168.144.10
10      192.168.100.9
11      192.168.100.4
dtype: object

# change source & destination ips to generated

In [15]:
mapping_dict = {key:generate_random_ipv4_address() if ('.' in key) and (key not in l_local_ips) 
                else generate_random_ipv6_address() if key not in l_local_ips 
                else key for key in l_unique_all}

In [16]:
mapping_dict['192.168.100.4']

'192.168.100.4'

In [17]:
ddf['Source'] = ddf['Source'].map(mapping_dict)
ddf['Destination'] = ddf['Destination'].map(mapping_dict)

In [22]:
ddf.head()

/opt/conda/lib/python3.10/site-packages/distributed/worker.py:2975: UserWarning: Large object of size 3.23 MiB detected in task graph: 
  ("('getitem-18660a1bc37300870a2a0ed1efde5b10', 0)" ... 45.128'}, None)
Consider scattering large objects ahead of time
with client.scatter to reduce scheduler burden and 
keep data on workers

    future = client.submit(func, big_data)    # bad

    big_future = client.scatter(big_data)     # good
    future = client.submit(func, big_future)  # good
  warnings.warn(


,No.,tcp length,tcp stream,udp stream,src port,dst port,timestamp,delta time,Source,Destination,Protocol,Length,Info,expert info
0,1,0.0,0.0,NaN,51413.0,19063.0,"2023-07-28 21:21:38,450421",0.000000,171.22.98.209,192.168.100.4,TCP,54,"51413 > 19063 [FIN, ACK] Seq=1 Ack=1 Win=409...",Note
1,2,0.0,0.0,NaN,19063.0,51413.0,"2023-07-28 21:21:38,450718",0.000297,192.168.100.4,171.22.98.209,TCP,54,19063 > 51413 [ACK] Seq=1 Ack=2 Win=512 Len=0,None
2,3,NaN,NaN,0.0,57621.0,57621.0,"2023-07-28 21:21:38,454832",0.004114,192.168.100.4,192.168.100.255,UDP,86,57621 > 57621 Len=44,None
3,4,0.0,1.0,NaN,34069.0,19081.0,"2023-07-28 21:21:38,455025",0.000193,135.62.170.152,192.168.100.4,TCP,54,"34069 > 19081 [RST, ACK] Seq=1 Ack=1 Win=0 L...",Warning
4,5,0.0,2.0,NaN,20482.0,19083.0,"2023-07-28 21:21:38,455025",0.000000,52.171.138.181,192.168.100.4,TCP,54,20482 > 19083 [ACK] Seq=1 Ack=1 Win=255 Len=0,None


# add column with partition id

In [18]:
def partition_id_column(df, partition_info = None):
    df['capture_id'] = partition_info['number']
    return df

In [19]:
types = ddf.dtypes.apply(str)
types['capture_id'] = 'int64'
types = list(zip(types.index,types))
types

[('No.', 'int64'),
 ('tcp length', 'float64'),
 ('tcp stream', 'float64'),
 ('udp stream', 'float64'),
 ('src port', 'float64'),
 ('dst port', 'float64'),
 ('timestamp', 'object'),
 ('delta time', 'float64'),
 ('Source', 'object'),
 ('Destination', 'object'),
 ('Protocol', 'object'),
 ('Length', 'int64'),
 ('Info', 'object'),
 ('expert info', 'object'),
 ('capture_id', 'int64')]

In [20]:
ddf = ddf.map_partitions(partition_id_column, meta = types)

In [36]:
ddf.head()

,No.,tcp length,tcp stream,udp stream,src port,dst port,timestamp,delta time,Source,Destination,Protocol,Length,Info,expert info,capture_id
0,1,0.0,0.0,NaN,51413.0,19063.0,"2023-07-28 21:21:38,450421",0.000000,171.22.98.209,192.168.100.4,TCP,54,"51413 > 19063 [FIN, ACK] Seq=1 Ack=1 Win=409...",Note,0
1,2,0.0,0.0,NaN,19063.0,51413.0,"2023-07-28 21:21:38,450718",0.000297,192.168.100.4,171.22.98.209,TCP,54,19063 > 51413 [ACK] Seq=1 Ack=2 Win=512 Len=0,None,0
2,3,NaN,NaN,0.0,57621.0,57621.0,"2023-07-28 21:21:38,454832",0.004114,192.168.100.4,192.168.100.255,UDP,86,57621 > 57621 Len=44,None,0
3,4,0.0,1.0,NaN,34069.0,19081.0,"2023-07-28 21:21:38,455025",0.000193,135.62.170.152,192.168.100.4,TCP,54,"34069 > 19081 [RST, ACK] Seq=1 Ack=1 Win=0 L...",Warning,0
4,5,0.0,2.0,NaN,20482.0,19083.0,"2023-07-28 21:21:38,455025",0.000000,52.171.138.181,192.168.100.4,TCP,54,20482 > 19083 [ACK] Seq=1 Ack=1 Win=255 Len=0,None,0


In [21]:
ddf.to_parquet('data/')

/opt/conda/lib/python3.10/site-packages/distributed/worker.py:2975: UserWarning: Large object of size 3.22 MiB detected in task graph: 
  ("('getitem-68f780c973f3f42bf66ff4d25c2ccb41', 0)" ... 8:69f3'}, None)
Consider scattering large objects ahead of time
with client.scatter to reduce scheduler burden and 
keep data on workers

    future = client.submit(func, big_data)    # bad

    big_future = client.scatter(big_data)     # good
    future = client.submit(func, big_future)  # good
  warnings.warn(


# repartition to 4x smaller

In [26]:
ddf = dd.read_parquet('data/part.*.parquet')

In [27]:
ddf = ddf.repartition(npartitions = 500)

In [28]:
ddf.to_parquet('data/4xsmaller/')

In [29]:
client.close()